### Get Coordinates of HDB flats, MRT locations and primary schools

In [2]:
import pandas as pd
import requests
import os

In [3]:
# test API call
import json
add = "504D YISHUN ST 51"
url= "https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text) 
data

{'found': 1,
 'totalNumPages': 1,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'ACACIA BREEZE @ YISHUN',
   'BLK_NO': '504D',
   'ROAD_NAME': 'YISHUN STREET 51',
   'BUILDING': 'ACACIA BREEZE @ YISHUN',
   'ADDRESS': '504D YISHUN STREET 51 ACACIA BREEZE @ YISHUN SINGAPORE 764504',
   'POSTAL': '764504',
   'X': '29157.7128495453',
   'Y': '44457.6450243499',
   'LATITUDE': '1.41833362541375',
   'LONGITUDE': '103.843721655882'}]}

In [4]:
# to get distinct locations from merged_df
merged_df = pd.read_csv('data/merged_df.csv')
location_df = merged_df[['block', 'street_name']].drop_duplicates().reset_index(drop=True)
location_df.shape

(9279, 2)

In [5]:
# function to get location details from OneMap API
def get_location_details(block, street):
    search_val = f"{block} {street}"
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal={search_val}"
    response = requests.get(url)
    data = json.loads(response.text)
    
    if data['found'] > 0:
        result = data['results'][0]
        return {
            'block': block,
            'street_name': street,
            'Address': result['ADDRESS'],
            'Postal': result['POSTAL'],
            'Latitude': float(result['LATITUDE']),
            'Longitude': float(result['LONGITUDE'])
        }
    return {
        'block': block,
        'street_name': street,
        'Address': None,
        'Postal': None,
        'Latitude': None,
        'Longitude': None
    }

In [ ]:
# create empty dataframe with desired columns
location_details = pd.DataFrame(columns=['block', 'street_name', 'Address', 'Postal', 'Latitude', 'Longitude'])

# iterate through unique locations and append to dataframe
for idx, row in location_df.iterrows():
    details = get_location_details(row['block'], row['street_name'])
    # Convert the dictionary to a DataFrame row
    new_row = pd.DataFrame([{
        'block': row['block'],
        'street_name': row['street_name'],
        'Address': details['Address'],
        'Postal': details['Postal'],
        'Latitude': details['Latitude'],
        'Longitude': details['Longitude']
    }])
    location_details = pd.concat([location_details, new_row], ignore_index=True)
    
    # Print progress every 10 locations
    if idx % 10 == 0:
        print(f"Processed {idx} locations...")

# Display first few rows of the dataframe
print("\nFirst few rows of location details:")
location_details.head()

In [ ]:
# save location details to csv
location_details.to_csv('data/hdb_locations.csv', index=False)

### Read in csv files which show locations of primary schools and MRT station locations

Both are extracted from https://github.com/hxchua/datadoubleconfirm/tree/master/datasets

In [6]:
mrt = pd.read_csv('data/mrtsg.csv')
primary_schools = pd.read_csv('data/primaryschoolsg.csv')

In [7]:
mrt.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [8]:
mrt[mrt['STN_NAME'] == 'RAFFLES PLACE MRT STATION']

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
134,186,RAFFLES PLACE MRT STATION,EW14,30019.1805,29617.6126,1.284126,103.851457,GREEN
135,186,RAFFLES PLACE MRT STATION,NS26,30019.1805,29617.6126,1.284126,103.851457,RED


#### To calculate distance between flat and nearby amenities (i.e. Primary school, MRT stations), and distance to CBD (defined by distance to Raffles Place MRT)

In [9]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r * 1000  # Convert to meters

In [ ]:
# create a new column for MRT count in location_details
location_details['mrt_count_500m'] = 0

# iterate through each HDB location
for idx, row in location_details.iterrows():
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        continue
        
    count = 0
    # check distance to each MRT station
    for _, mrt_row in mrt.iterrows():
        distance = haversine_distance(
            row['Latitude'], 
            row['Longitude'],
            mrt_row['Latitude'],
            mrt_row['Longitude']
        )
        if distance <= 500:  # add count if within 500 meters
            count += 1
    
    # store the count in the new column
    location_details.at[idx, 'mrt_count_500m'] = count

### Count number of schools and top 25 primary schools within 2km of HDB block

In [ ]:
# create new columns for school counts
location_details['schools_within_2km'] = 0
location_details['top25_schools_within_2km'] = 0
location_details['nearby_top25_schools'] = '' 


# get list of top 25 schools from primary_school dataframe
top25_schools = primary_schools[primary_schools['top25sch'] == 1]['Name'].tolist()

# iterate through each HDB location
for idx, row in location_details.iterrows():
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        continue
        
    total_count = 0
    top25_count = 0
    nearby_top25_list = []

    
    # check distance to each school
    for _, school_row in primary_schools.iterrows():
        distance = haversine_distance(
            row['Latitude'], 
            row['Longitude'],
            school_row['Latitude'],
            school_row['Longitude']
        )
        
        if distance <= 2000:  # If within 2km
            total_count += 1
            # check if it's a top 25 school using the top25sch column
            if school_row['top25sch'] == 1:
                top25_count += 1
                nearby_top25_list.append(school_row['Name'])
    
    # store the counts in the new columns
    location_details.at[idx, 'schools_within_2km'] = total_count
    location_details.at[idx, 'top25_schools_within_2km'] = top25_count
    location_details.at[idx, 'nearby_top25_schools'] = ', '.join(nearby_top25_list) 

In [30]:
# Raffles Place MRT coordinates
RAFFLES_LAT = 1.284125611
RAFFLES_LON = 103.8514572

# Calculate distance to Raffles Place MRT for each HDB block
location_details['distance_to_cbd'] = location_details.apply(
    lambda row: haversine_distance(
        row['Latitude'],
        row['Longitude'],
        RAFFLES_LAT,
        RAFFLES_LON
    ) if pd.notna(row['Latitude']) and pd.notna(row['Longitude']) else None,
    axis=1
)

In [31]:
location_details.head()

,block,street_name,Address,Postal,Latitude,Longitude,mrt_count_500m,schools_within_2km,top25_schools_within_2km,nearby_top25_schools,distance_to_cbd
0,309,ANG MO KIO AVE 1,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,560207,1.365821,103.842848,0,10,4,"Ai Tong School, Catholic High School (Primary)...",9134.378661
1,216,ANG MO KIO AVE 1,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,560216,1.366197,103.841505,0,10,3,"Ai Tong School, Catholic High School (Primary)...",9192.714790
2,211,ANG MO KIO AVE 3,211 ANG MO KIO AVENUE 3 SINGAPORE 560211,560211,1.369197,103.841667,0,11,4,"Ai Tong School, Catholic High School (Primary)...",9521.908686
3,202,ANG MO KIO AVE 3,202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAP...,560202,1.368446,103.844516,0,11,4,"Ai Tong School, Catholic High School (Primary)...",9407.743380
4,235,ANG MO KIO AVE 3,235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...,560235,1.366824,103.836491,0,8,3,"Ai Tong School, Catholic High School (Primary)...",9344.886133


In [ ]:
# save the updated location details to csv file
location_details.to_csv('data/location_details.csv', index=False)